## Simple Chatbot using Python and Deep Learning

This is a work-in-progress chatbot with a simple but growing dictionary.

guide by Kurtis Pykes: 
https://towardsdatascience.com/a-simple-chatbot-in-python-with-deep-learning-3e8669997758

see also: https://towardsdatascience.com/how-to-build-your-own-chatbot-using-deep-learning-bb41f970e281

#### Installing needed packages in virtual environment:

In [1]:
!which jupyter

/Users/danielk/opt/anaconda3/envs/dl/bin/jupyter


In [2]:
!conda install tensorflow -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.10.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [3]:
!pip install nltk

You should consider upgrading via the '/Users/danielk/opt/anaconda3/envs/dl/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install colorama

You should consider upgrading via the '/Users/danielk/opt/anaconda3/envs/dl/bin/python -m pip install --upgrade pip' command.


#### Importing libraries:

In [5]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
from colorama import Fore, Style
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /Users/danielk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/danielk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Dictionary:

In [6]:
# used a dictionary to represent an intents JSON file
data = {"intents": [
    {"tag": "greeting",
     "patterns": ["Hallo", "Hi", "Aloha", "Wie geht's?", "Grüß dich"],
     "responses": ["Howdy!", "Hallo", "Wie geht's?", "Grüß dich!", "Na, alles fit?", "Wunderschönen guten Tag!"],
    },
    {"tag": "age",
     "patterns": ["wie alt bist du?", "wann hast du Geburtstag?", "wann wurdest du geboren?"],
     "responses": ["Ich bin 24", "Ich wurde vor 24 Jahren geboren", "Am 23. Dezember werde ich 25", "Das ist geheim."]
    },
    {"tag": "why",
     "patterns": ["warum?", "wieso?", "weshalb?"],
     "responses": ["kann ich dir nicht sagen", "das weiß niemand so wirklich", "Darauf habe ich keine Antwort"]
    },
    {"tag": "thanks",
     "patterns": ["Danke", "Vielen Dank", "Das ist hilfreich", "Danke für die Hilfe"],
     "responses": ["Kein Thema!", "Gerne!", "Mit Vergnügen", "Gern geschehen!"]
    },
    {"tag": "about",
     "patterns": ["Wer bist du?", "Was bist du?", "Was machst du?" ],
     "responses": ["Ich bin Rüdiger, dein freundlicher chatbot", "Ich Rüdiger, eine beschränkte KI"]
    },
    {"tag": "date",
     "patterns": ["was machst du am Wochenende",
                  "wollen wir mal abhängen?", "hast du Pläne diese Woche?"],
     "responses": ["Ich habe die ganze Woche Zeit", "Ich habe noch keine Pläne", "Ich hab frei"]
    },
    {"tag": "name",
     "patterns": ["was ist dein Name?", "wie heißt du?", "wie nennt man dich?"],
     "responses": ["Mein Name ist Rüdiger", "Ich bin Rüdiger", "Rüdiger"]
    },
    {"tag": "goodbye",
     "patterns": [ "bye", "tschüss", "bis bald", "hat mich gefreut", "man sieht sich", "mach's gut"],
     "responses": ["War mir eine Freude", "Adios", "Bis bald!", "Mach's gut!"]
    }
]}

In [7]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

#### Training Data:

In [8]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

#### Model:

In [9]:
# from tensorflow.keras.callbacks import EarlyStopping

In [10]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])

# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               7296      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 520       
Total params: 16,072
Trainable params: 16,072
Non-trainable params: 0
_________________________________________________________________
None
Train on 30 samples
Epoch 1/200
30/30 [==============================] - 2s 58ms/sample - loss: 2.1054 - accuracy

Epoch 70/200
30/30 [==============================] - 0s 445us/sample - loss: 7.6560e-04 - accuracy: 1.0000
Epoch 71/200
30/30 [==============================] - 0s 760us/sample - loss: 0.0067 - accuracy: 1.0000
Epoch 72/200
30/30 [==============================] - 0s 620us/sample - loss: 0.0013 - accuracy: 1.0000
Epoch 73/200
30/30 [==============================] - 0s 404us/sample - loss: 0.0027 - accuracy: 1.0000
Epoch 74/200
30/30 [==============================] - 0s 816us/sample - loss: 0.0135 - accuracy: 1.0000
Epoch 75/200
30/30 [==============================] - 0s 474us/sample - loss: 0.0095 - accuracy: 1.0000
Epoch 76/200
30/30 [==============================] - 0s 665us/sample - loss: 0.0045 - accuracy: 1.0000
Epoch 77/200
30/30 [==============================] - 0s 435us/sample - loss: 0.0016 - accuracy: 1.0000
Epoch 78/200
30/30 [==============================] - 0s 674us/sample - loss: 4.9948e-04 - accuracy: 1.0000
Epoch 79/200
30/30 [==============================] - 0s

Epoch 148/200
30/30 [==============================] - 0s 542us/sample - loss: 4.1846e-04 - accuracy: 1.0000
Epoch 149/200
30/30 [==============================] - 0s 484us/sample - loss: 0.0025 - accuracy: 1.0000
Epoch 150/200
30/30 [==============================] - 0s 435us/sample - loss: 8.2257e-04 - accuracy: 1.0000
Epoch 151/200
30/30 [==============================] - 0s 356us/sample - loss: 1.1695e-04 - accuracy: 1.0000
Epoch 152/200
30/30 [==============================] - 0s 458us/sample - loss: 7.4293e-04 - accuracy: 1.0000
Epoch 153/200
30/30 [==============================] - 0s 406us/sample - loss: 4.9891e-05 - accuracy: 1.0000
Epoch 154/200
30/30 [==============================] - 0s 587us/sample - loss: 0.0090 - accuracy: 1.0000
Epoch 155/200
30/30 [==============================] - 0s 812us/sample - loss: 0.1193 - accuracy: 0.9333
Epoch 156/200
30/30 [==============================] - 0s 531us/sample - loss: 0.0012 - accuracy: 1.0000
Epoch 157/200
30/30 [==============

#### Create Features:

In [11]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

#### Running the Chatbot:

In [16]:
print(Fore.YELLOW + "Beginne eine Konversation (schreibe EXIT zum Verlassen):" + Style.RESET_ALL)
while True:
    print(Fore.LIGHTBLUE_EX + "Du: " + Style.RESET_ALL, end='')
    message = input("")
    if message == "exit":
        break
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(Fore.GREEN + "Bot:" + Style.RESET_ALL, result)

Beginne eine Konversation (schreibe EXIT zum Verlassen):
Du: hi
Bot: Wie geht's?
Du: gut und dir?
Bot: Bis bald!
Du: exit
